In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import entropy
import datetime
import copy
import time
import wandb
import pickle as pkl
import sys
sys.path.append('../../')
from data.processing import get_data
import models
from main.seir.fitting import single_fitting_cycle
from main.seir.forecast import get_forecast, forecast_all_trials, create_all_trials_csv, create_decile_csv_new
from main.seir.sensitivity import calculate_sensitivity_and_plot
from utils.generic.create_report import save_dict_and_create_report
from utils.generic.config import read_config
from utils.generic.enums import Columns
from utils.fitting.loss import Loss_Calculator
from utils.generic.logging import log_wandb
from viz import plot_forecast, plot_top_k_trials, plot_ptiles
from viz.fit import plot_histogram, plot_all_histograms, plot_mean_variance, plot_scatter, plot_kl_divergence, plot_heatmap_distribution_sigmas, plot_all_params, plot_all_losses, plot_all_buckets, plot_cv_in_params, plot_recovery_loss, plot_confidence_interval
import yaml
from data.dataloader import SimulatedDataLoader

In [ ]:
simulated_config_filename = 'seirhd_fixed.yaml'
with open(os.path.join("../../configs/simulated_data/", simulated_config_filename)) as configfile:
    simulated_config = yaml.load(configfile, Loader=yaml.SafeLoader)    
actual_params = simulated_config['params']

In [ ]:
predictions_dict = {}

In [ ]:
config_filename = 'default.yaml'
config = read_config(config_filename)

In [ ]:
n = 2
m = 2
scenario_dict = {}
for i in range(1,n+1):
    best_fit_params = []
    for j in range(1,m+1):
        predictions_dict['m' + str(i)] = single_fitting_cycle(**copy.deepcopy(config['fitting'])) 
        best_fit_params.append(predictions_dict['m' + str(j)]['best_params'])
    scenario_dict['s' + str(i)] = best_fit_params

In [ ]:
fig, axs = plt.subplots(nrows=8//2, ncols=2, 
                            figsize=(18, 6*8//2))
plot_histogram(predictions_dict['m1'], fig, axs, weighting='exp', beta=1, plot_lines=False, weighted=True, 
                   savefig=False, filename=None, label=None)

In [ ]:
fig, axs = plt.subplots(nrows=8//2, ncols=2, 
                            figsize=(18, 6*8//2))
plot_confidence_interval(scenario_dict, fig, axs, actual_params=actual_params, ranges=config['fitting']['variable_param_ranges'])